In [3]:
# !pip install yfinance crewai crewai-tools

In [18]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool

In [5]:
from pydantic import BaseModel, Field

class QueryAnalysisOutput(BaseModel):
    """Structured output for the query analysis task."""
    symbol: str = Field(..., description="Stock ticker symbol (e.g., TSLA, AAPL).")
    timeframe: str = Field(..., description="Time period (e.g., '1d', '1mo', '1y').")
    action: str = Field(..., description="Action to be performed (e.g., 'fetch', 'plot').")

In [9]:
llm = LLM(
    model="ollama/deepseek-r1:7b",
    base_url="http://localhost:11434",
    # temperature=0.7
)

# 1) Query parser agent
query_parser_agent = Agent(
    role="Stock Data Analyst",
    goal="Extract stock details and fetch required data from this user query: {query}.",
    backstory="You are a financial analyst specializing in stock market data retrieval.",
    llm=llm,
    verbose=True,
    memory=True,
)

query_parsing_task = Task(
    description="Analyze the user query and extract stock details.",
    expected_output="A dictionary with keys: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)


# 2) Code writer agent
code_writer_agent = Agent(
    role="Senior Python Developer",
    goal="Write Python code to visualize stock data.",
    backstory="""You are a Senior Python developer specializing in stock market data visualization. 
                 You are also a Pandas, Matplotlib and yfinance library expert.
                 You are skilled at writing production-ready Python code""",
    llm=llm,
    verbose=True,
)

code_writer_task = Task(
    description="""Write Python code to visualize stock data based on the inputs from the stock analyst
                   where you would find stock symbol, timeframe and action.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_writer_agent,
)


# 3) Code interpreter agent (uses code interpreter tool from crewai)
code_interpreter_tool = CodeInterpreterTool()

code_execution_agent = Agent(
    role="Senior Code Execution Expert",
    goal="Review and execute the generated Python code by code writer agent to visualize stock data.",
    backstory="You are a code execution expert. You are skilled at executing Python code.",
    # tools=[code_interpreter_tool],
    allow_code_execution=True,   # This automatically adds the CodeInterpreterTool
    llm=llm,
    verbose=True,
)

code_execution_task = Task(
    description="""Review and execute the generated Python code by code writer agent to visualize stock data.""",
    expected_output="A clean and executable Python script file (.py) for stock visualization.",
    agent=code_execution_agent,
)


In [10]:
### --- CREW SETUP --- ###

crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

# Run the crew with an example query
result = crew.kickoff(inputs={"query": "Plot YTD stock gain of Tesla"})

# Agent: Stock Data Analyst
## Task: Analyze the user query and extract stock details.


# Agent: Stock Data Analyst
## Final Answer: 
<think>
Alright, let me try to figure out what I need to do here. The user has given me a task as a stock data analyst where I have to extract specific details from their query. The query is "Plot YTD stock gain of Tesla." 

First, I'll break down the query into parts to understand each component. The main goal is to plot (create a graph) of the Year-to-Date (YTD) stock gain for Tesla.

Looking at this, there are three key elements that stand out:
1. **Symbol**: This should be the ticker symbol for Tesla. From what I remember, Tesla's ticker is TSLA.
2. **Timeframe**: The query mentions "YTD," which stands for Year-to-Date, so the timeframe is definitely YTD.
3. **Action**: The user wants a plot or graph of the stock gain over this period.

Now, considering these elements:
- **Symbol**: TSLA
- **Timeframe**: YTD
- **Action**: Plot (which refers to gener

In [16]:
print(result.raw)

```python
# This Python code will plot YTD stock data for Tesla using historical prices from Yahoo Finance
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Extracting the required information from the JSON configuration
config = {
    "symbol": "TSLA",
    "timeframe": "YTD",
    "action": "Plot"
}

# Fetching historical data for Tesla from Yahoo Finance
tesla_data = yfinance.download(
    tickers=config['symbol'],
    period=config['timeframe'],
    interval='1d'
)

# Creating a simple plot of the stock prices over time
plt.figure(figsize=(10, 6))
plt.plot(tesla_data.index, tesla_data['Adj Close'])
plt.title(f'Tesla Stock Price (TSLA) - {config['timeframe']}')
plt.xlabel('Date')
plt.ylabel('Adjusted Closing Price ($)')
plt.grid(True)
plt.show()
```
